In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as pt
import json

In [3]:
jsonFile= open('projectData/resolvedIssues.json', 'r')
jsonRawData= jsonFile.read()
jsonData = json.loads(jsonRawData)
# issues = jsonData['issues']
issues = pd.DataFrame(jsonData['issues'])
# issues['key']
# with open('projectData/buggs.txt', 'w') as test:
#     for line in issues['key']:
#         test.write(line+"\n")

In [4]:
gitLog= open('projectData/issueList.json', 'r')
gitData = json.load(gitLog)
filteredDict = {k:v for k,v in gitData.items() if k[:6]=='HADOOP'}
gitIssues = pd.Series(list(filteredDict.keys()))

a=[]
for x in issues['key']:
    for y in gitIssues:
        if x==y:
            a.append(x)

finalGitDict= {k:v for k,v in filteredDict.items() if k in a}
finalGitDict

# with open("projectData/filteredGitLog.json", 'w+') as f:
#     json.dump(finalGitDict, f)
    

{'HADOOP-12302': [['e2d46ac5a4b50a19ba1075c6b32147e81088d6ec',
   'e92107b18f82b3501deaa6170d322a0fb512ec71']],
 'HADOOP-12268': [['b9ec7f15d03933aa2538d8c764f71045399111ad',
   '52ee3ecdf683c6963c0e1a8e7567067320950d08']],
 'HADOOP-10945': [['e6b4714d0b87a0fcefe1994fcc34fc88001c2486',
   '6f50a6a4565d89a886a240c459899692ed5807c3']],
 'HADOOP-12175': [['587593566914a0a4333c9ba87e59fc6488aec122',
   '2ec008eb38b7a8b4dceb5e64c4c6d73fee147740']],
 'HADOOP-12237': [['621203bf44886f21e4aa93071ad87b461ecfcc92',
   'f54862ff154575c0f185efdeb9ba83cfe3cdf856']],
 'HADOOP-12202': [['f54862ff154575c0f185efdeb9ba83cfe3cdf856',
   'a6cad7d856421a18398bdac9ca52f8c01101b8a4']],
 'HADOOP-11797': [['2a0688fdb820ff1563198e546717cb5d3cb93adc',
   'd2ee02940aede8b86c6cddce416f89bbac0e2980']],
 'HADOOP-12239': [['bb080b32d46fe02544d36b91de794860dd7d55f3',
   '930e344d7885cc392eb3ee01e59b748f175fff21']],
 'HADOOP-12017': [['03d68b55755f6b8c6f035656199d551ff549efd0',
   '53ed25c3d19b4357c65da7d8ac61386077183

In [125]:
import pickle
df = pd.read_pickle("projectData/authorExperienceMatrix.p")
dF = pd.DataFrame(df.items())
dF.columns= ["name", "experience"]
dF[['names','commits']]= pd.DataFrame(dF.name.tolist(), index=dF.index)

In [126]:
from pandas.io.json import json_normalize

In [147]:
from collections import namedtuple
AuthorCommitTuple = namedtuple('AuthorCommitTuple', ('author', 'commit'))
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_colwidth', -1)
df = pd.read_pickle("projectData/defectAuthorRelation.p")
dF = pd.DataFrame(df.items(), columns=['commits', 'data'])
print(pd.DataFrame(dF['data'].items()))

     0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [111]:
# dF[['names','commits']]= pd.DataFrame(dF.name.tolist(), index=dF.index)